## **SRTF Code**

In [1]:
sample_processes=[ #Sample dataset with pid, arrival time, burst time, and priority (can add more PIDs or the values can be changed)
    {"pid": "P1", "arrival": 0, "burst": 10, "priority": 4},
    {"pid": "P2", "arrival": 1, "burst": 6, "priority": 3},
    {"pid": "P3", "arrival": 2, "burst": 11, "priority": 5},
    {"pid": "P4", "arrival": 3, "burst": 7, "priority": 4}
]

def print_schedule(results): #Function to print scheduling results in a table
    print(f"{'PID':<5} {'Start':<6} {'Finish':<6} {'Waiting':<7} {'Turnaround':<10}") #Print the table with fixed column widths
    for r in results: #Loop through each result entry (each process)
        print(f"{r['pid']:<5} {r['start']:<6} {r['finish']:<6} "
              f"{r['waiting']:<7} {r['turnaround']:<10}")

def srtf(sample_processes):
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes] #Add "remaining" field to track remaining burst
    time = 0 #Current simulation time
    complete = 0 #Number of completed processes
    n = len(sample_processes) #Total number of processes
    results = {} #Stores results sorted by PID
    current = None #Currently running process

    while complete < n: #Continue until all processes finish
        available = [p for p in sample_processes if p["arrival"] <= time and p["remaining"] > 0] #All arrived processes with remaining CPU time

        if not available: # If nothing has arrived yet or all are finished, advance time until something becomes available
            time += 1
            continue

        p = min(available, key=lambda x: x["remaining"]) #Choose process with the "shortest remaining time"

        if p["pid"] not in results: #First time this process gets CPU
            results[p["pid"]] = {"pid": p["pid"], "start": time} #Its first start time (could be preempted later)

        p["remaining"] -= 1 #Execute process for 1 unit of time (preemptive)
        time += 1 #Move the time time forward

        if p["remaining"] == 0: #If process has finished
            results[p["pid"]]["finish"] = time #Record finish time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"]) #Waiting time = finish - arrival - burst
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"]) #Turnaround time = finish - arrival
            complete += 1

    return list(results.values()) #Convert dict to list
    
print("SRTF:")
print_schedule(srtf(sample_processes))

SRTF:
PID   Start  Finish Waiting Turnaround
P1    0      23     13      23        
P2    1      7      0       6         
P4    7      14     4       11        
P3    23     34     21      32        


## **Gantt Chart & Averages Code**

In [3]:
def make_gantt_chart(results):
    results = sorted(results, key=lambda x: x["start"]) #Sort tasks by start time for correct chart order

    chart = "" #For bar representation
    timeline = "" #For timestamp markers

    for r in results: #Build bars and timeline for each process
        length = r["finish"] - r["start"] #Duration of process execution
        bar = "-" * length
        chart += f"|{bar}"
        timeline += f"{r['start']}{' ' * length}" #Add start time and spacing

    timeline += f"{results[-1]['finish']}" #Add final ending time

    print("\nGantt Chart:")
    print(chart + "|")

    label_line = ""
    for r in results: #Create a line with PID labels centered in each bar
        length = r["finish"] - r["start"]
        label_line += "|" + r["pid"].center(length, "-") #Center PID inside the block
    print(label_line + "|")

    print(timeline) #Print timeline with time markers

def compute_averages(results): #Compute average waiting & turnaround times
    total_wait = sum(r["waiting"] for r in results) #Sum of all waiting times
    total_turn = sum(r["turnaround"] for r in results) #Sum of all turnaround times
    n = len(results) #Number of processes
    return {"avg_waiting": total_wait / n, "avg_turnaround": total_turn / n} # Average waiting and turnaround time

results = srtf(sample_processes)
make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])


Gantt Chart:
|-----------------------|------|-------|-----------|
|-----------P1----------|--P2--|---P4--|-----P3----|
0                       1      7       23           34

Average Waiting Time = 9.5
Average Turnaround Time = 18.0
